In [7]:
import pandas as pd
import torchtext
from torchtext.vocab import Vectors
from torchtext.legacy import data
import torch
from torch import nn
import torch.nn.functional as F
from d2l import torch as d2l
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.autograd import Variable
from sklearn import metrics
import sys
sys.path.append("D:/Experiment")
from tqdm import tqdm
from MyKu import training
from MyKu import processing
from torchtext.vocab import Vectors
from spacy.lang.en import English

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(DEVICE)


cuda:0


In [4]:
processing.create_OLID()
# processing.set_olid_train_data(processing.ORIGIN_DATASET_PATH + '/OLID')
# processing.set_olid_testA_data(processing.ORIGIN_DATASET_PATH + '/OLID')


In [3]:
# train_data = processing.get_OLID_train_data(processing.OLID_DATASET + '/train.tsv')
# test_data = processing.get_OLID_testA_data(
#     processing.OLID_DATASET + '/testA.tsv', processing.OLID_DATASET + '/labels-levela.csv')


In [8]:
spacy_en = English()

def tokenizer(text):  # create a tokenizer function
    """
    定义分词操作
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

def DataLoader():
    def tokenize(x): return x.split()

    TEXT = data.Field(sequential=True, tokenize=tokenizer, lower=True, include_lengths=True)
    LABEL = data.Field(sequential=False, use_vocab=False)
    # 假如train.csv文件并不是只有两列，比如1、3列是review和polarity，2列是我们不需要的数据，
    # 那么就要添加一个全是None的元组， fields列表存储的Field的顺序必须和csv文件中每一列的顺序对应，

    train_fields = [(None, None), ('tweet', TEXT), ('subtask_a', LABEL)]
    # train_fields = [(None, None), (None, None), (None, None), (None, None), ('text', TEXT), ('task1', LABEL)]
    train_data= data.TabularDataset(
    path='D:/Experiment/datasets/OLID/train.tsv',
    # path='D:/Experiment/datasets/EXIST2021/train.tsv',
    format='tsv',
    fields=train_fields,
    skip_header=True  # 是否跳过文件的第一行
    )
    test_fields = [(None, None), ('tweet', TEXT), ('label', LABEL)]
    # test_fields = [(None, None), (None, None), (None, None), (None, None), ('text', TEXT), ('task1', LABEL)]
    test_data= data.TabularDataset(
    path='D:/Experiment/datasets/OLID/testA.tsv',
    # path='D:/Experiment/datasets/EXIST2021/test.tsv',
    format='tsv',
    fields=test_fields,
    skip_header=True  # 是否跳过文件的第一行
    )
    return train_data, test_data, TEXT, LABEL


In [9]:
train_data, test_data, TEXT, LABEL = DataLoader()
# x1, x2 = 1, 1
# for index in train_data:
#     index.task1 = 1 if index.task1 == 'sexist' else 0

# for index in test_data:
#     index.task1 = 1 if index.task1 == 'sexist' else 0

# temp_examples = []
# for index in train_data.examples:
#     if len(index.text) == 0:
#         continue
#     if x1 > 3437:
#         break
#     else:
#         x1 += 1
#     temp_examples.append(index)
# train_data.examples = temp_examples
# temp_examples = []
# for index in test_data.examples:
#     if len(index.text) == 0:
#         continue
#     if x2 > 1000:
#         break
#     else:
#         x2 += 1
#     temp_examples.append(index)
# test_data.examples = temp_examples

vectors = Vectors(name='glove.6B.300d.txt', cache=processing.EMBEDDING_PATH)

TEXT.build_vocab(train_data,  # 建词表是用训练集建，不要用验证集和测试集
                  max_size=400000, # 单词表容量
                  vectors=vectors, # 还有'glove.840B.300d'已经很多可以选
                  unk_init=torch.Tensor.normal_ # 初始化train_data中不存在预训练词向量词表中的单词
)

In [10]:
# vectors.stoi['apples']
# vectors.itos[253]
train_data[0].tweet


['@user',
 'she',
 'should',
 'ask',
 'a',
 'few',
 'native',
 'americans',
 'what',
 'their',
 'take',
 'on',
 'this',
 'is',
 '.']

In [13]:
# class BiRNN(nn.Module):
#     def __init__(self, vocab_size, embed_size, num_hiddens, num_layers, **kwargs):
#         super(BiRNN, self).__init__(**kwargs)
#         self.embedding = nn.Embedding(vocab_size, embed_size)
#         self.encoder = nn.LSTM(
#             embed_size, num_hiddens, num_layers=num_layers, bidirectional=True, dropout=0.8)
#         self.decoder = nn.Linear(num_hiddens * 4, 2)

#     def forward(self, inputs):
#         embeddings = self.embedding(inputs)
#         # print(embeddings.shape)
#         self.encoder.flatten_parameters()
#         outputs, _ = self.encoder(embeddings)
#         # print(outputs.shape)
#         encoding = torch.cat((outputs[0], outputs[-1]), dim=1)
#         outs = self.decoder(encoding)
#         # print(outs.shape)
#         return outs

class BiLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_size, num_hidden, output_dim, num_layers, dropout, pad_idx):
        super(BiLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_size, padding_idx=pad_idx)
        self.lstm = nn.LSTM(embedding_size, num_hidden, num_layers=num_layers, bidirectional=True, batch_first=True)
        self.num_layers = num_layers
        self.num_hiddens = num_hidden
        # self.dropout = nn.Dropout(dropout)
        # self.fc = nn.Linear(num_hidden * 2, num_hidden)
        # self.relu = nn.ReLU()
        self.classifier = nn.Linear(num_hidden * 2, output_dim)


    def init_hidden(self, batch_size):
        h, c = (Variable(torch.zeros(self.num_layers * 2, batch_size, self.num_hiddens)),
            Variable(torch.zeros(self.num_layers * 2, batch_size, self.num_hiddens)))
        return h, c

    def forward(self, text, text_len):
        batch_size = text.shape[1]
        h_0, c_0 = self.init_hidden(batch_size)
        h_0, c_0 = h_0.to(DEVICE), c_0.to(DEVICE)
        embedded = self.embedding(text.permute(1, 0))
        # print('embedded size', embedded.shape)
        packed_embedded = pack_padded_sequence(embedded, text_len.cpu(), batch_first=True)
        output_packed, (h_n, c_n) = self.lstm(packed_embedded, (h_0, c_0))
        # print('output_packed size', output_packed.data.shape)
        output_unpacked, output_len = pad_packed_sequence(output_packed, batch_first=True)
        # print('output_unpacked size', output_unpacked.data.shape)
        output = output_unpacked[:, -1, :]
        # rel = self.relu(output)
        # dence = self.fc(rel)
        # drop = self.dropout(dence)
        preds = self.classifier(output)
        # preds = F.softmax(preds, dim=1)
        return preds

# example shape
# text size torch.Size([11, 32])
# embedded size torch.Size([32, 11, 300])
# batch size 32
# output_packed size torch.Size([352, 200])
# output_unpacked size torch.Size([32, 11, 200])
# torch.Size([32, 2])

In [14]:
# X = torch.ones((4, 6), dtype=torch.long).to(device=DEVICE)
model = BiLSTM(len(TEXT.vocab), 300, 200, 2, 2, 0.5, 1)
model.to(DEVICE)

BiLSTM(
  (embedding): Embedding(20335, 300, padding_idx=1)
  (lstm): LSTM(300, 200, num_layers=2, batch_first=True, bidirectional=True)
  (classifier): Linear(in_features=400, out_features=2, bias=True)
)

In [6]:
pretrained_embeddings = TEXT.vocab.vectors
# print(pretrained_embeddings.shape)
model.embedding.weight.data.copy_(pretrained_embeddings)
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(300)
model.embedding.weight.data[PAD_IDX] = torch.zeros(300)

In [7]:
train_iter, test_iter = data.BucketIterator.splits(
    (train_data, test_data), batch_size=64, sort_within_batch=True, sort_key=lambda x : len(x.tweet), device=DEVICE
)

In [8]:
def train(model, train_iter, optimizer, loss, epoch):
    model.train()
    epoch_loss = 0
    num_sample = 0
    correct = 0
    for batch in tqdm(train_iter, desc=f"Training Epoch {epoch}", colour='red'):
        optimizer.zero_grad()
        text, text_len = batch.tweet
        label = batch.subtask_a
        # text, text_len = batch.text
        # label = batch.task1
        # print(text_len.shape)
        output = model(text, text_len)
        pred_y = torch.argmax(output, dim=1)
        correct += torch.sum(pred_y == label)
        l = loss(output, label)
        l.backward()
        epoch_loss += l.item()
        num_sample += len(batch)
        optimizer.step()
    print(
        f'\tTrain Loss: {epoch_loss / num_sample:.3f} | Train Acc: {correct.float() / num_sample* 100:.2f}%')


def test(model, test_iter):
    true_y, pred_y = [], []
    for batch in tqdm(test_iter, desc=f"Testing", colour='green'):
        text, text_len = batch.tweet
        label = batch.label
        # text, text_len = batch.text
        # label = batch.task1
        with torch.no_grad():
            output = model(text, text_len)
            pred_y.extend(output.argmax(dim=1).tolist())
            true_y.extend(label.tolist())
    print(metrics.confusion_matrix(true_y, pred_y))
    print(metrics.classification_report(true_y, pred_y))
    print(f'Acc : {metrics.accuracy_score(true_y, pred_y)}\t F1: {metrics.f1_score(true_y, pred_y, average="macro")}')


In [15]:
lr, num_epochs = 0.0001, 10
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss = nn.CrossEntropyLoss()
for epoch in range(1, num_epochs+1):
    train(model, train_iter, optimizer, loss, epoch)
    test(model, test_iter)

Training Epoch 1: 100%|██████████| 207/207 [00:04<00:00, 50.29it/s]


	Train Loss: 0.010 | Train Acc: 62.16%


Testing: 100%|██████████| 14/14 [00:00<00:00, 131.95it/s]
d:\Software\anaconda3\envs\pytorch_env\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\Software\anaconda3\envs\pytorch_env\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\Software\anaconda3\envs\pytorch_env\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_pr

[[620   0]
 [240   0]]
              precision    recall  f1-score   support

           0       0.72      1.00      0.84       620
           1       0.00      0.00      0.00       240

    accuracy                           0.72       860
   macro avg       0.36      0.50      0.42       860
weighted avg       0.52      0.72      0.60       860

Acc : 0.7209302325581395	 F1: 0.4189189189189189


Training Epoch 2: 100%|██████████| 207/207 [00:03<00:00, 53.30it/s]


...

[[600  20]
 [228  12]]
              precision    recall  f1-score   support

           0       0.72      0.97      0.83       620
           1       0.38      0.05      0.09       240

    accuracy                           0.71       860
   macro avg       0.55      0.51      0.46       860
weighted avg       0.63      0.71      0.62       860

Acc : 0.7116279069767442	 F1: 0.4584822879428014


Training Epoch 10: 100%|██████████| 207/207 [00:03<00:00, 53.51it/s]


	Train Loss: 0.006 | Train Acc: 81.16%


Testing: 100%|██████████| 14/14 [00:00<00:00, 124.89it/s]

[[604  16]
 [230  10]]
              precision    recall  f1-score   support

           0       0.72      0.97      0.83       620
           1       0.38      0.04      0.08       240

    accuracy                           0.71       860
   macro avg       0.55      0.51      0.45       860
weighted avg       0.63      0.71      0.62       860

Acc : 0.713953488372093	 F1: 0.4529997621288434
